# PETs/TETs – Hyperledger Aries – Authority Agent (Issuing Authority) 🏛️


In [1]:
%%javascript
document.title='🏛️ Authority Agent'

<IPython.core.display.Javascript object>

## PART 2: Issue a VC to the Manufacturer Agents

**What:** Issue verifiable credentials (VCs) to all manufacturers

**Why:** Manufacturers will be able to store VCs, and prove to the city (the data scientist) that they are manufacturers, without revealing their identity.

**How:** <br>
1. [Initiate Authority's AgentCommunicationManager (ACM)](#1) <br>
2. [Connect with Manufacturer1](#2)
3. [Issue VC to Manufacturer1](#3)
4. [Repeat Steps 2-3 for Manufacturer2](#4)
4. [Repeat Steps 2-3 for Manufacturer3](#5)

**Accompanying Agents and Notebooks:** 
* Manufacturer1 💼: `02_get_manufacturer1_VC.ipynb`
* Manufacturer2 💼: `02_get_manufacturer2_VC.ipynb`
* Manufacturer3 💼: `02_get_manufacturer3_VC.ipynb`

---

### 0 - Setup
#### 0.1 - Imports

In [1]:
from aries_cloudcontroller import AriesAgentController
import libs.helpers as helpers
from libs.agent_connection_manager import IssuingAuthority
import os
from termcolor import colored

#### 0.2 – Variables

In [2]:
# Get identifier data defined in notebook 00_init_authority_as_issuingAuthority.ipynb
identifiers = helpers.get_identifiers()
schema_manufacturer_id = identifiers["manufacturer_schema_identifiers"]["schema_id"]
cred_def_manufacturer_id = identifiers["manufacturer_schema_identifiers"]["cred_def"]

# Get environment variables
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

---

<a id=1> </a>
### 1 – Initiate Authority Agent

#### 1.1 – Init ACA-PY agent controller

In [3]:
# Setup
agent_controller = AriesAgentController(admin_url,api_key)
print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")

Initialising a controller with admin api at http://authority-agent:3021 and an api key of adminApiKey


#### 1.2 – Start Webhook Server to enable communication with other agents
@todo: is communication with other agents, or with other docker containers?

In [4]:
# Listen on webhook server
await agent_controller.init_webhook_server(webhook_host, webhook_port)
print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


#### 1.3 – Init ACM issuing authority

In [5]:
# The IssuingAuthority registers relevant webhook servers and event listeners
authority_agent = IssuingAuthority(agent_controller)

Successfully initiated AgentConnectionManager for a(n) Issuing Authority ACA-PY agent


---

<a id=2> </a>

### 2 – Establish a connection with Manufacturer1
A connection with the credential issuer (i.e., the authority agent) must be established before a VC can be received. In this scenario, the manufacturer1 requests a connection with the Authority to be certified as an official city agency. Thus, the manufacturer1 agent sends an invitation to the Authority. In real life, the invitation can be shared via video call, phone call, or E-Mail. In this PoC, this is represented by copy and pasting the invitation into the manufacturers' notebooks.

### 2.1 – Receive invitation from `Manufacturer1` agent
Copy the invitation from Step 2.1 in the City's `01_hold_VC.ipynb` notebook into the following cell.
Several state changes of the connection between the Manufacturer agent, the inviter (A), and the authority agent, the invitee (B),  are required before successfully establishing a connection: 

| Step | State | Agent | Description | Function/Prompt/Variable |
| --- | --- | --- | --- | --- |
| 1 | invitation-sent | A | A sent an invitation to B | `create_connection_invitation()`
| 2 | invitation-received | B | B receives the invitation of A | Prompt: Paste invitation from A |
| 3 | request-sent | B | B sends B connection request | Prompt: Accept invitation OR `auto_accept=True` |
| 4 | request-received | A | A receives the connection request from B | Prompt: Accept invitation request response OR `auto_accept=True` |
| 5 | response-sent | A | A sends B response to B | - |
| 6 | response-received | B | B receives the response from A | - |
| 7 | active (completed) | A | B pings A to finalize connection | Prompt: Trust ping OR `auto_ping=True` |


In [6]:
# Variables
alias = None
auto_accept= True
label = None

# Receive connection invitation
connection_id_m1 = authority_agent.receive_connection_invitation(alias=alias, auto_accept=auto_accept, label=label)

Please enter invitation received by external agent.


Invitation:  {     '@id': 'cdeea3cf-77b3-4acb-b731-a199bf6f5deb',     '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',     'label': 'Manufacturer1',     'recipientKeys': ['FJrxHFHaZTKHh3281UnwyTVpy66PMYvmjUnEfBy3FWSF'],     'serviceEndpoint': 'https://d07863f66b36.ngrok.io', }



---------------------------------------------------------------------
Connection Webhook Event Received
Connection ID :  56df1855-48f6-4990-a1fd-46a02aedc35b
State :  invitation (invitation-received)
Routing State : none
Connection with :  Manufacturer1
Their Role :  inviter
---------------------------------------------------------------------

---------------------------------------------------------------------
Connection Webhook Event Received
Connection ID :  56df1855-48f6-4990-a1fd-46a02aedc35b
State :  request (request-sent)
Routing State : none
Connection with :  Manufacturer1
Their Role :  inviter
---------------------------------------------------------------------

---------------------------------------------------------------------
Connection Webhook Event Received
Connection ID :  56df1855-48f6-4990-a1fd-46a02aedc35b
State :  response (response-received)
Routing State : none
Connection with :  Manufacturer1
Their Role :  inviter
-------------------------------------------

<div style="font-size: 25px"><center><b>Break Point 2</b></center></div>
<div style="font-size: 50px"><center>🏛 ➡️ 💼</center></div><br>
<center><b>Please return to the Manufacturer1's notebook 💼. <br>Check the prompts in Step 2.1 (e.g., if auto_accept or auto_ping are set to False), and then proceed to Step 3.</b></center>

---

<a id=3> </a>
## 3 – Process VC request by Manufacturers
### 3.1 – Check messages / requests by Manufacturers
Check inbox and await messages sent by Manufacturer1

In [7]:
# Verify inbox
message_ids = authority_agent.verify_inbox()


---------------------------------------------------------------------
Message Inbox
> 2 Message(s) via Connection ID 56df1855-48f6-4990-a1fd-46a02aedc35b:
	 * Message ID :  4dabef14-a041-41c9-b0fe-66b4e84e62cc
	 * Message ID :  2600eae6-9856-4a2d-abd7-767b623d5907
---------------------------------------------------------------------


In [8]:
for m_id in message_ids:
    authority_agent.get_message(m_id)


---------------------------------------------------------------------
Message received
Connection ID :  56df1855-48f6-4990-a1fd-46a02aedc35b
Message ID :  4dabef14-a041-41c9-b0fe-66b4e84e62cc
State :  received
Time :  2021-08-18 06:49:25.214010Z
Text :  Hello Authority. I would like to be certified as a manufacturer.
---------------------------------------------------------------------

---------------------------------------------------------------------
Message received
Connection ID :  56df1855-48f6-4990-a1fd-46a02aedc35b
Message ID :  2600eae6-9856-4a2d-abd7-767b623d5907
State :  received
Time :  2021-08-18 06:49:25.426051Z
Text :  {"manufacturerName": "undisclosedName1", "manufacturerCountry": "DE", "manufacturerCity": "Berlin"}
---------------------------------------------------------------------


### 3.3 – Offer VC to `Manufacturer1` agent
The next step is to offer a VC to the manufacturer agent. The manufacturer can then request the offer and store it in their wallet. The following table provides an overview of the individual states between I (Issuer, the Authority agent) and H (Holder, the Manufacturer).

| Step | State | Role | Description | Function/Prompt/Variable |
| --- | --- | --- | --- | --- |
| 1 | offer_sent | I | I sends I VC offer with personalized information to H| `offer_vc()` |
| 2 | offer_received | H | H receives offer made by I | - |
| 3 | request_sent | H | Request VC offer | `request_vc()` AND (Prompt: request VC OR `auto_request=True`) |
| 4 | request_received | I | M1's request to get VC was received | - |
| 5 | credential_issued | I | Automatic response to issue credential | - |
| 6 | credential_received | H| H receives VC and is asked to store it | Prompt: Store VC OR `auto_store=True`
| 7 | credential_acked | I / H | Credential was issued and stored | - |

In [9]:
# MAKE VC ZKP-able! SEE https://github.com/hyperledger/aries-cloudagent-python/blob/main/JsonLdCredentials.md
comment = "Issuing VC that Manufacturer1 is a manufacturer"
auto_remove = True
trace = False

# Offer Manufacturer1 a VC with manufacturer_schema
authority_agent.offer_vc(
    connection_id_m1, 
    schema_manufacturer_id, 
    cred_def_manufacturer_id, 
    comment=comment, 
    # Comment out next line if you do not want to get the prompts to enter VC information
    credential_attributes=[{"name": "manufacturerName", "value": "undisclosedManufacturer1"}, {"name": "manufacturerCity", "value": "Berlin"}, {"name": "manufacturerCountry", "value": "Germany"}, {"name": "isManufacturer", "value": "TRUE"}]
)


---------------------------------------------------------------------
Handle Issue Credential Webhook
Connection ID : 56df1855-48f6-4990-a1fd-46a02aedc35b
Credential exchange ID : 46183706-034e-4daf-accb-957a4a58fe55
Agent Protocol Role :  issuer
Protocol State :  offer_sent
Proposed Credential : 
{
    '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview',
    'attributes': [
        {'name': 'manufacturerName', 'value': 'undisclosedManufacturer1'},
        {'name': 'manufacturerCity', 'value': 'Berlin'},
        {'name': 'manufacturerCountry', 'value': 'Germany'},
        {'name': 'isManufacturer', 'value': 'TRUE'},
    ],
}
---------------------------------------------------------------------

---------------------------------------------------------------------
Handle Issue Credential Webhook
Connection ID : 56df1855-48f6-4990-a1fd-46a02aedc35b
Credential exchange ID : 46183706-034e-4daf-accb-957a4a58fe55
Agent Protocol Role :  issuer
Protocol Stat

<div style="font-size: 25px"><center><b>Break Point 4</b></center></div>
<div style="font-size: 50px"><center>🏛 ➡️ 💼</center></div><br>
<center><b>Please return to the Manufacturer1's notebook 💼. <br>Continue with Step 3.2</b></center>

---

<a id=4> </a>
### 4 – Repeat Steps 2 and 3 with Manufacturer2 🙈
Execute the following cells to certify Manufacturer2 that the agent is a manufacturer.
#### 4.1 – Receive connection invitation by Manufacturer2

In [10]:
# Variables
alias = None
auto_accept= True
label = None

# Receive connection invitation
connection_id_m2= authority_agent.receive_connection_invitation(alias=alias, auto_accept=auto_accept, label=label)

Please enter invitation received by external agent.


Invitation:  {     '@id': '538124f5-ecd8-4bf0-b86c-fcf1c14d8755',     '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',     'label': 'manufacturer2',     'recipientKeys': ['DESCQQFisH4mMgEgJbvVtztmYoYgAnAHrFNGQuTAd39M'],     'serviceEndpoint': 'https://d8f72f418259.ngrok.io', }



---------------------------------------------------------------------
Connection Webhook Event Received
Connection ID :  cb475bc3-8a2c-46da-b597-691b22896e7f
State :  invitation (invitation-received)
Routing State : none
Connection with :  manufacturer2
Their Role :  inviter
---------------------------------------------------------------------

---------------------------------------------------------------------
Connection Webhook Event Received
Connection ID :  cb475bc3-8a2c-46da-b597-691b22896e7f
State :  request (request-sent)
Routing State : none
Connection with :  manufacturer2
Their Role :  inviter
---------------------------------------------------------------------

---------------------------------------------------------------------
Connection Webhook Event Received
Connection ID :  cb475bc3-8a2c-46da-b597-691b22896e7f
State :  response (response-received)
Routing State : none
Connection with :  manufacturer2
Their Role :  inviter
-------------------------------------------

#### 4.2 Offer VC to Manufacturer2

In [11]:
# MAKE VC ZKP-able! SEE https://github.com/hyperledger/aries-cloudagent-python/blob/main/JsonLdCredentials.md
comment = "Issuing VC that Manufacturer2 is a manufacturer"
auto_remove = True
trace = False

# Offer Manufacturer1 a VC with manufacturer_schema
authority_agent.offer_vc(
    connection_id_m2, 
    schema_manufacturer_id, 
    cred_def_manufacturer_id, 
    comment=comment, 
    # Comment out next line if you do not want to get the prompts to enter VC information
    credential_attributes=[{"name": "manufacturerName", "value": "undisclosedManufacturer2"}, {"name": "manufacturerCity", "value": "Braunschweig"}, {"name": "manufacturerCountry", "value": "Germany"}, {"name": "isManufacturer", "value": "TRUE"}]
)


---------------------------------------------------------------------
Handle Issue Credential Webhook
Connection ID : cb475bc3-8a2c-46da-b597-691b22896e7f
Credential exchange ID : a93f28eb-7a47-42c3-9612-e9b977a1e6b1
Agent Protocol Role :  issuer
Protocol State :  offer_sent
Proposed Credential : 
{
    '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview',
    'attributes': [
        {'name': 'manufacturerName', 'value': 'undisclosedManufacturer2'},
        {'name': 'manufacturerCity', 'value': 'Braunschweig'},
        {'name': 'manufacturerCountry', 'value': 'Germany'},
        {'name': 'isManufacturer', 'value': 'TRUE'},
    ],
}
---------------------------------------------------------------------

---------------------------------------------------------------------
Handle Issue Credential Webhook
Connection ID : cb475bc3-8a2c-46da-b597-691b22896e7f
Credential exchange ID : a93f28eb-7a47-42c3-9612-e9b977a1e6b1
Agent Protocol Role :  issuer
Protoco

<div style="font-size: 25px"><center><b>Break Point 6</b></center></div>
<div style="font-size: 50px"><center>🏛 ➡️ 💼</center></div><br>
<center><b>Please return to the Manufacturer2's notebook 💼. <br>Continue with Step 3.1</b></center>

---

<a id=5> </a>
### 5 – Repeat Steps 2 and 3 with Manufacturer3
Execute the following cells to certify Manufacturer3 that the agent is a manufacturer.
#### 5.1 – Establish a connection with Manufacturer3

In [12]:
# Variables
alias = None
auto_accept= True
label = None

# Receive connection invitation
connection_id_m3 = authority_agent.receive_connection_invitation(alias=alias, auto_accept=auto_accept, label=label)

Please enter invitation received by external agent.


Invitation:  {     '@id': '8293e29e-6c55-4134-97cd-5fa4ad8b7f9d',     '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',     'label': 'manufacturer3',     'recipientKeys': ['DMtrnUz2VM6EtRKMpLA2NTNmo2NdnwLN5TkN6aNVFdFW'],     'serviceEndpoint': 'https://0b29bf65469e.ngrok.io', }



---------------------------------------------------------------------
Connection Webhook Event Received
Connection ID :  74b2f02f-41a0-432a-b6ca-014413b1b7cd
State :  invitation (invitation-received)
Routing State : none
Connection with :  manufacturer3
Their Role :  inviter
---------------------------------------------------------------------

---------------------------------------------------------------------
Connection Webhook Event Received
Connection ID :  74b2f02f-41a0-432a-b6ca-014413b1b7cd
State :  request (request-sent)
Routing State : none
Connection with :  manufacturer3
Their Role :  inviter
---------------------------------------------------------------------

---------------------------------------------------------------------
Connection Webhook Event Received
Connection ID :  74b2f02f-41a0-432a-b6ca-014413b1b7cd
State :  response (response-received)
Routing State : none
Connection with :  manufacturer3
Their Role :  inviter
-------------------------------------------

#### 5.2 Offer VC to Manufacturer2

In [13]:
# MAKE VC ZKP-able! SEE https://github.com/hyperledger/aries-cloudagent-python/blob/main/JsonLdCredentials.md
comment = "Issuing VC that Manufacturer3 is a manufacturer"
auto_remove = True
trace = False

# Offer Manufacturer1 a VC with manufacturer_schema
authority_agent.offer_vc(
    connection_id_m3, 
    schema_manufacturer_id, 
    cred_def_manufacturer_id, 
    comment=comment, 
    # Comment out next line if you do not want to get the prompts to enter VC information
    credential_attributes=[{"name": "manufacturerName", "value": "undisclosedManufacturer3"}, {"name": "manufacturerCity", "value": "München"}, {"name": "manufacturerCountry", "value": "Germany"}, {"name": "isManufacturer", "value": "TRUE"}]
)


---------------------------------------------------------------------
Handle Issue Credential Webhook
Connection ID : 74b2f02f-41a0-432a-b6ca-014413b1b7cd
Credential exchange ID : 8bb1ce1f-76cb-4948-874a-ccea4cd1fe0f
Agent Protocol Role :  issuer
Protocol State :  offer_sent
Proposed Credential : 
{
    '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview',
    'attributes': [
        {'name': 'manufacturerName', 'value': 'undisclosedManufacturer3'},
        {'name': 'manufacturerCity', 'value': 'München'},
        {'name': 'manufacturerCountry', 'value': 'Germany'},
        {'name': 'isManufacturer', 'value': 'TRUE'},
    ],
}
---------------------------------------------------------------------

---------------------------------------------------------------------
Handle Issue Credential Webhook
Connection ID : 74b2f02f-41a0-432a-b6ca-014413b1b7cd
Credential exchange ID : 8bb1ce1f-76cb-4948-874a-ccea4cd1fe0f
Agent Protocol Role :  issuer
Protocol Sta

<div style="font-size: 25px"><center><b>Break Point 9</b></center></div>
<div style="font-size: 50px"><center>🏛 ➡️ 💼</center></div><br>
<center><b>Please return to the Manufacturer3's notebook 💼. <br>Continue with Step 3.1</b></center>

---

## 6 - Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [14]:
await agent_controller.terminate()

--- 

### 🔥🔥🔥 You are done 🙌 and can close this notebook now 🔥🔥🔥
